In [ ]:
# ================= 安装依赖 =================
!pip install flask pyngrok

# ================= 导入模块 =================
import subprocess, os, signal, time, threading, requests
from datetime import datetime
from flask import Flask, send_from_directory
from pyngrok import ngrok

# ================= 配置参数 =================
REMOTE_URL = "https://kong.pp.ua/a.txt"  # ← 你只需要修改这一行
HLS_OUTPUT = "stream/stream.m3u8"
PID_FILE = "ffmpeg.pid"
LOG_FILE = "ffmpeg.log"
NGROK_TOKEN = "2vztoTvKNpetesaYNhaWUFM9bUt_5vZPxULruLkB1NVACT7UA"  # ← 自己去 ngrok.com 申请一个免费账号
CHECK_INTERVAL = 30

# ================= 日志工具 =================
def log(msg):
    now = datetime.now().strftime('%H:%M:%S')
    print(f"[{now}] {msg}")

# ================= 函数逻辑 =================
def get_remote_url():
    try:
        return requests.get(REMOTE_URL, timeout=10).text.strip()
    except:
        log("获取 a.txt 失败")
        return None

def kill_ffmpeg():
    if os.path.exists(PID_FILE):
        try:
            with open(PID_FILE, 'r') as f:
                os.kill(int(f.read().strip()), signal.SIGKILL)
                log("FFmpeg 已终止")
        except:
            log("终止 FFmpeg 失败")
        os.remove(PID_FILE)

def start_ffmpeg(url):
    kill_ffmpeg()
    os.makedirs("stream", exist_ok=True)
    cmd = [
        "ffmpeg", "-user_agent", "Mozilla/5.0",
        "-i", url, "-c", "copy", "-sn",
        "-f", "hls", "-hls_time", "5",
        "-hls_list_size", "6",
        "-hls_flags", "delete_segments+append_list",
        HLS_OUTPUT
    ]
    with open(LOG_FILE, 'a') as f:
        p = subprocess.Popen(cmd, stdout=f, stderr=f)
    with open(PID_FILE, 'w') as f:
        f.write(str(p.pid))
    log(f"启动 FFmpeg：{url} PID={p.pid}")

def clean_stream_folder():
    if os.path.exists("stream"):
        for file in os.listdir("stream"):
            path = os.path.join("stream", file)
            if os.path.isfile(path) and time.time() - os.path.getmtime(path) > 180:
                os.remove(path)
                log(f"删除旧文件：{file}")

def loop():
    last_url = None
    while True:
        current_url = get_remote_url()
        if current_url and current_url.startswith("http") and current_url != last_url:
            log("检测到新链接，重启 FFmpeg")
            start_ffmpeg(current_url)
            last_url = current_url
        else:
            log("无变化或链接无效")
        clean_stream_folder()
        time.sleep(CHECK_INTERVAL)

# ================= 搭建 Web 控制台 =================
app = Flask(__name__)
@app.route('/stream/<path:path>')
def stream_file(path):
    return send_from_directory("stream", path)

def run_flask():
    app.run(port=8000)

# ================= 启动服务 =================
ngrok.set_auth_token(NGROK_TOKEN)
public_url = ngrok.connect(8000)
log(f"🎮 控制面板地址：{public_url}/stream/stream.m3u8")

threading.Thread(target=loop).start()
threading.Thread(target=run_flask).start()
